In [1]:
import pandas as pd
import datetime as dt
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [2]:
#def call_exchangerate(event=None,context=None):
#Converting datetime to String for easy access
x = dt.datetime.now().strftime("%m/%d/%Y") 

In [3]:
#Generate Start Date and End Date...
start_Day = 16
end_Day = 15
month = int(x.split('/')[0])  # To split month     


In [6]:
#Formating Start Date and End Date in DD/MM/YYYY format

if month == 1:
    #For Loading February month exchange rate in January, start month has to be start from previeous December
    start_date = str(int(x.split('/')[2])-1)+'/'+str(month+11)+'/'+str(start_Day)
    end_date = str(x.split('/')[2])+'/'+str(month)+'/'+str(end_Day)
    eff_year = int(x.split('/')[2])
    eff_month = 2
    
elif month == 12:
    #For Loading January month exchange rate in December, start month has to be start from previeous November
    start_date = str(int(x.split('/')[2])-1)+'/'+str(month-1)+'/'+str(start_Day)
    end_date = str(int(x.split('/')[2])-1)+'/'+str(month)+'/'+str(end_Day)
    eff_year = str(int(x.split('/')[2])+1)
    eff_month = 1

else:
    #For Other than January & February month in calendar year below condition has to be executed
    start_date = str(int(x.split('/')[2]))+'/'+str(month-1)+'/'+str(start_Day)
    end_date = str(x.split('/')[2])+'/'+str(month)+'/'+str(end_Day)
    eff_year = str(x.split('/')[2])
    eff_month = int(month) + 1
    print("Start_date: {},End_date: {},Effective_Year: {},Effective_Month: {}".format(start_date,end_date,eff_year,eff_month))

Start_date: 2023/8/16,End_date: 2023/9/15,Effective_Year: 2023,Effective_Month: 10


In [ ]:
#Defining Currency code in lists
gpma_cur = ['HKD','TWD','PLN','JPY','CNY','KRW','EUR','THB','USD','VND']
#['USD','JPY','CNY','HKD','KRW','TWD','PLN','EUR','THB','VND']
y_df = pd.DataFrame()
for x in gpma_cur:
    #Generating Dynamic URL framing using Generated Start & End Date rang
    url_df="http://www.oanda.com/rates/api/v1/rates/"+x+".json?api_key=gpma1203&decimal_places=9&start="+start_date+"&end="+end_date+"&fields=averages&quote=HKD&quote=TWD&quote=PLN&quote=JPY&quote=CNY&quote=KRW&quote=EUR&quote=THB&quote=USD&quote=VND"
    y_df = pd.DataFrame(y_df.append(pd.read_json(url_df)))

In [ ]:
#Deleting the unnecessary row from the dataframe
y_df.drop(index=['effective_params','request_time','skipped_currencies'],inplace=True)

In [ ]:
#Deleting the unnecessary column from the dataframe
y_df.drop(columns='meta',inplace=True)

In [ ]:
#Setting up Index as new column "conv_to_cur_code" 
y_df['conv_to_cur_code'] = y_df.index

In [ ]:
#Creating Data source column on dataframe
y_df['data_srce_id'] = 1


In [ ]:
#Creating new columns ask_price and bid_price on dataframe
y_df['ask_price'] = pd.Series()
y_df['bid_price'] = pd.Series()

In [ ]:
#Creating new Columns Effective Month and  Effective Year
y_df['Effective_Year'] = eff_year
y_df['Effective_Month'] = eff_month

In [ ]:
#Realigning Columns name
y_df = y_df[['data_srce_id','base_currency','conv_to_cur_code','ask_price','bid_price','Effective_Year','Effective_Month','quotes']]
y_df.rename(columns={'data_srce_id':'Data_SourceID','base_currency':'Base_Currency','conv_to_cur_code':'To_Currency',
                         'ask_price':'ASK_Price','bid_price':'BID_Price','quotes':'Quotes'},inplace=True)  

In [ ]:
#Feeding existing dictionary column data from quotes columns to "ask_price" & "bid_price" on data frame
for t in range(0,len(y_df)):
    if (y_df['Base_Currency'][t] == 'JPY') or (y_df['To_Currency'][t] == 'JPY'):
        a = float(y_df['Quotes'][t]['ask'])
        b = str(int(a))
        c = str(a - int(a))[1:9]
        ap = b + c
        #print("Base_currency:{},To_currency:{},Ask Price:{},row_count:{}".format(y_df['Base_Currency'][t],y_df['To_Currency'][t],float(ap),t))
        y_df['ASK_Price'][t] = float(ap)
        d = float(y_df['Quotes'][t]['bid'])
        e = str(int(d))
        f = str(d - int(d))[1:9]
        bp = e + f
        y_df['BID_Price'][t] = float(bp)
        
        #Below else if condition created becuase for Base currency VND to EUR & USD conversion values is 0 so i type cast it to Integer  
    elif (y_df['Base_Currency'][t] == 'VND' and (y_df['To_Currency'][t] == 'EUR'or y_df['To_Currency'][t] == 'USD')):
        y_df['ASK_Price'][t] = 0.0000
        y_df['BID_Price'][t] = 0.0000
            
    else:
        a = float(y_df['Quotes'][t]['ask'])
        b = str(int(a))
        c = str(a - int(a))[1:6]
        ap = b + c
        #print("Base_currency:{},To_currency:{},Ask Price:{},row_count:{}".format(y_df['Base_Currency'][t],y_df['To_Currency'][t],float(ap),t))
        y_df['ASK_Price'][t] = float(ap)
        d = float(y_df['Quotes'][t]['bid'])
        e = str(int(d))
        f = str(d - int(d))[1:6]
        bp = e + f
        y_df['BID_Price'][t] = float(bp)

In [ ]:
# Create the plain-text and HTML version of your message
text = y_df.to_html()

In [ ]:
# Turn these into plain/html MIMEText objects
part1 = MIMEText(text, "html")

In [ ]:
#Triggering email with the content of dataframe as HTML to receptents
sender_email = "mailsivacs@gmail.com"
receiver_email = ["snatarajan5@mmm.com","mailsivacs@gmail.com"]
password = input("Type your password and press enter:")

message = MIMEMultipart("alternative")
message["Subject"] = "OANDA Exchange Rates"
message["From"] = sender_email
message["To"] = ",".join(receiver_email)

In [ ]:
# Create the plain-text and HTML version of your message
text = y_df.to_html()

In [ ]:
# Turn these into plain/html MIMEText objects
part1 = MIMEText(text, "html")


In [ ]:
# Add HTML/plain-text parts to MIMEMultipart message
# The email client will try to render the last part first
message.attach(part1)


In [ ]:
# Create secure connection with server and send email
context = ssl.create_default_context() 
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, message.as_string()) 